In [1]:
import plotly.express as px
import re

In [2]:
import plotly.io as pio
pio.renderers.default = "browser"

In [3]:
from collections import OrderedDict
import argparse
import re
from scipy.stats import gmean

pattern_map = {
    'runtime': 'simTicks\s+\d+',
    'network_lat': 'system.ruby.network.average_packet_network_latency\s+\d+',
    'inst': 'simInsts\s+\d+',
    'd_rd_access': 'system.switch_cpus\d+.mmu.dtb.rdAccesses\s+\d+',
    'i_rd_access': 'system.switch_cpus\d+.mmu.itb.rdAccesses\s+\d+',
    'd_wr_access': 'system.switch_cpus\d+.mmu.dtb.wrAccesses\s+\d+',
    'i_wr_access': 'system.switch_cpus\d+.mmu.itb.wrAccesses\s+\d+',
    'd_rd_miss': 'system.switch_cpus\d+.mmu.dtb.rdMisses\s+\d+',
    'i_rd_miss': 'system.switch_cpus\d+.mmu.itb.rdMisses\s+\d+',
    'd_wr_miss': 'system.switch_cpus\d+.mmu.dtb.wrMisses\s+\d+',
    'i_wr_miss': 'system.switch_cpus\d+.mmu.itb.wrMisses\s+\d+',
    
    'sd_rd_0': 'system.ruby.l2_cntrl\d+.L2cache.readLinearHist::0-\d+\s+',
    'sd_t': 'system.ruby.l2_cntrl\d+.L2cache.m_demand_accesses\s+\d+',
    'sd_inf': 'system.ruby.l2_cntrl\d+.L2cache.infiniteSD\s+\d+',
    'bin_size_rd': 'system.ruby.l2_cntrl\d+.L2cache.readLinearHist::0-\d+\s+',
    
    'sd_wr_0': 'system.ruby.l2_cntrl\d+.L2cache.writeLinearHist::0-\d+\s+',
    'bin_size_wr': 'system.ruby.l2_cntrl\d+.L2cache.writeLinearHist::0-\d+\s+',
}

prune_map = {
    'runtime': lambda line: float(line.replace(' ','').split('simTicks')[-1].split('#')[0]),
    'network_lat': lambda line: float(line.replace(' ','').split('system.ruby.network.average_packet_network_latency')[-1].split('(')[0]),
    'inst': lambda line: float(line.replace(' ','').split('simInsts')[-1].split('#')[0]),
    'd_rd_access': lambda line: float(line.replace(' ','').split('Accesses')[-1].split('#')[0]),
    'i_rd_access': lambda line: float(line.replace(' ','').split('Accesses')[-1].split('#')[0]),
    'd_wr_access': lambda line: float(line.replace(' ','').split('Accesses')[-1].split('#')[0]),
    'i_wr_access': lambda line: float(line.replace(' ','').split('Accesses')[-1].split('#')[0]),
    'd_rd_miss': lambda line: float(line.replace(' ','').split('Misses')[-1].split('#')[0]),
    'i_rd_miss': lambda line: float(line.replace(' ','').split('Misses')[-1].split('#')[0]),
    'd_wr_miss': lambda line: float(line.replace(' ','').split('Misses')[-1].split('#')[0]),
    'i_wr_miss': lambda line: float(line.replace(' ','').split('Misses')[-1].split('#')[0]),
    'sd_rd_0': lambda line: int(re.sub('system.ruby.l2_cntrl\d+.L2cache.readLinearHist::0-\d+\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'sd_t': lambda line: int(line.replace(' ','').split('demand_accesses')[-1].split('#')[0]),
    'sd_inf': lambda line: int(line.replace(' ','').split('infiniteSD')[-1].split('#')[0].split('(')[0]),
    'bin_size_rd': lambda line: int(re.sub('system.ruby.l2_cntrl\d+.L2cache.readLinearHist::0-', '',line)\
                                    .split(' ')[0])+1,
    
    'sd_wr_0': lambda line: int(re.sub('system.ruby.l2_cntrl\d+.L2cache.writeLinearHist::0-\d+\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'bin_size_wr': lambda line: int(re.sub('system.ruby.l2_cntrl\d+.L2cache.writeLinearHist::0-', '',line)\
                                    .split(' ')[0])+1,
}


In [4]:
# perf results
time_str = '230630'
bench = ['blackscholes', 'bodytrack', 'canneal', 'dedup', \
         'ferret', 'fluidanimate', 'freqmine', 'streamcluster', 'swaptions', \
        'raytrace', 'vips']
sim_str = 'simlarge'
config = [0, 1, 2]
outdir = [f'{time_str}/gem5/my_STATS/base/{b}_{sim_str}_64_{c}/stats.txt' for b in bench for c in config ]
# outdir 


In [5]:
# hist results
time_str = '20220709083259'
bench = ['blackscholes',\
         'bodytrack', \
#          'canneal', \
#          'dedup', \
#          'ferret', 
         'fluidanimate', \
#          'freqmine', 'streamcluster', 
        'swaptions', \
        'raytrace', \
#          'vips'\
        ]
sim_str = 'simlarge'
config = [2]
outdir_hist = [f'{time_str}/{b}_{sim_str}_64_{c}/stats.txt' for b in bench for c in config ]


In [6]:
def get_list(file, metric):
    pattern = pattern_map[metric]

    list_ = []
    with open(file, 'r') as searchfile:
        for line in searchfile:
            if re.match(pattern, line):
                prune_func = prune_map[metric]
                res = prune_func(line)
                list_.append(res)
#     print(list_)
    return list_


In [7]:
o_arr = []
for o in outdir:
    name=o.split('/')[4].split('_')[0]
    if name not in o_arr: o_arr.append(name)
o_arr

['blackscholes',
 'bodytrack',
 'canneal',
 'dedup',
 'ferret',
 'fluidanimate',
 'freqmine',
 'streamcluster',
 'swaptions',
 'raytrace',
 'vips']

In [8]:
metric = 'runtime'
rt_list_base = [get_list(o, metric)[0] for o in outdir if '_1' in o]
print(f'rt base ={rt_list_base}')
rt_list_quater = [get_list(o, metric)[0] for o in outdir if '_0' in o]
print(f'rt quater={rt_list_quater}')
rt_list_red = [get_list(o, metric)[0] for o in outdir if '_2' in o]
print(f'rt red={rt_list_red}\n')

rt_norm_quater = [q / b for b,q in zip(rt_list_base, rt_list_quater)]
rt_norm_mean_quater = gmean(rt_norm_quater)
print(f'rt (norm) quater ={rt_norm_quater}')

rt_norm_red = [q / b for b,q in zip(rt_list_base, rt_list_red)]
rt_norm_mean_red = gmean(rt_norm_red)
print(f'rt (norm) red ={rt_norm_red}')

rt base =[90309490001.0, 1450780229501.0, 226178607001.0, 11387584559501.0, 4188090637001.0, 480568578501.0, 9981278638001.0, 5247092623001.0, 869625321501.0, 284651481501.0, 1361267624001.0]
rt quater=[90432085001.0, 1466946156001.0, 271252456501.0, 10828894044001.0, 4251534547501.0, 481092752001.0, 10054861853001.0, 5406488826501.0, 872739820501.0, 290982047001.0, 1429791260501.0]
rt red=[89988769501.0, 1450982830501.0, 267988688501.0, 11020174489001.0, 4223973999001.0, 474881881501.0, 10011050171501.0, 5403236867501.0, 871666482501.0, 290498174501.0, 298196218500.0]

rt (norm) quater =[1.0013574985308702, 1.0111429189420098, 1.1992843182547355, 0.9509386285932017, 1.0151486479159464, 1.0010907361060413, 1.0073721231185604, 1.0303780045355546, 1.0035814263027947, 1.0222397068394593, 1.0503381078722767]
rt (norm) red =[0.9964486511883032, 1.0001396496835842, 1.1848542709426766, 0.9677359084729291, 1.0085679525851177, 0.9881667315459157, 1.0029827374407376, 1.0297582405569763, 1.002347

In [9]:
import plotly.graph_objects as go

o_arr.append('geomean')
rt_norm_quater.append(rt_norm_mean_quater)
rt_norm_red.append(rt_norm_mean_red)

fig = go.Figure(data=[
    go.Bar(name='quater', x=o_arr, y=rt_norm_quater),
    go.Bar(name='fastAccess', x=o_arr, y=rt_norm_red)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()


In [10]:
#//////// rd
metric = 'd_rd_access'
d_rd_access_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'd rd access base ={d_rd_access_base}')
d_rd_access_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'd rd access quater={d_rd_access_quater}')
d_rd_access_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'd rd access red={d_rd_access_red}\n')

metric = 'd_rd_miss'
d_rd_miss_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'd rd miss base ={d_rd_miss_base}')
# d_rd_mr_base = [float(m/a)*100 for m,a in zip(d_rd_miss_base, d_rd_access_base)]
# print(f'r rd mr base={d_rd_mr_base}')

d_rd_miss_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'd rd miss quater={d_rd_miss_quater}')
# d_rd_mr_quater = [float(m/a)*100 for m,a in zip(d_rd_miss_quater, d_rd_access_quater)]
# print(f'r rd mr quater={d_rd_mr_quater}')

d_rd_miss_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'd rd miss red={d_rd_miss_red}\n')
# d_rd_mr_red = [float(m/a)*100 for m,a in zip(d_rd_miss_red, d_rd_access_red)]
# print(f'r rd mr red={d_rd_mr_red}')

#//////// wr
metric = 'd_wr_access'
d_wr_access_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'd wr access base ={d_wr_access_base}')
d_wr_access_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'd wr access quater={d_wr_access_quater}')
d_wr_access_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'd wr access red={d_wr_access_red}\n')

metric = 'd_wr_miss'
d_wr_miss_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'd wr miss base ={d_wr_miss_base}')
# d_wr_mr_base = [float(m/a)*100 for m,a in zip(d_wr_miss_base, d_wr_access_base)]
# print(f'r wr mr base={d_wr_mr_base}')

d_wr_miss_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'd wr miss quater={d_wr_miss_quater}')
# d_wr_mr_quater = [float(m/a)*100 for m,a in zip(d_wr_miss_quater, d_wr_access_quater)]
# print(f'r wr mr quater={d_rd_wr_quater}')

d_wr_miss_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'd wr miss red={d_wr_miss_red}\n')
# d_wr_mr_red = [float(m/a)*100 for m,a in zip(d_wr_miss_red, d_wr_access_red)]
# print(f'r wr mr red={d_wr_mr_red}')


In [11]:
#//////// rd
metric = 'i_rd_access'
i_rd_access_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'i rd access base ={i_rd_access_base}')
i_rd_access_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'i rd access quater={i_rd_access_quater}')
i_rd_access_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'i rd access red={i_rd_access_red}\n')

metric = 'i_rd_miss'
i_rd_miss_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'i rd miss base ={i_rd_miss_base}')
# i_rd_mr_base = [float(m/a)*100 for m,a in zip(i_rd_miss_base, i_rd_access_base)]
# print(f'i rd mr base={i_rd_mr_base}')

i_rd_miss_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'i rd miss quater={i_rd_miss_quater}')
# i_rd_mr_quater = [float(m/a)*100 for m,a in zip(i_rd_miss_quater, i_rd_access_quater)]
# print(f'i rd mr quater={i_rd_mr_quater}')

i_rd_miss_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'd rd miss red={i_rd_miss_red}\n')
# i_rd_mr_red = [float(m/a)*100 for m,a in zip(i_rd_miss_red, i_rd_access_red)]
# print(f'i rd mr red={i_rd_mr_red}')

#//////// wr
metric = 'i_wr_access'
i_wr_access_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'd wr access base ={i_wr_access_base}')
i_wr_access_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'd wr access quater={i_wr_access_quater}')
i_wr_access_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'd wr access red={i_wr_access_red}\n')

metric = 'i_wr_miss'
i_wr_miss_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'i wr miss base ={i_wr_miss_base}')
# i_wr_mr_base = [float(m/a)*100 for m,a in zip(i_wr_miss_base, i_wr_access_base)]
# print(f'i wr mr base={i_wr_mr_base}')

i_wr_miss_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'i wr miss quater={i_wr_miss_quater}')
# i_wr_mr_quater = [float(m/a)*100 for m,a in zip(i_wr_miss_quater, i_wr_access_quater)]
# print(f'i wr mr quater={i_rd_wr_quater}')

i_wr_miss_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'i wr miss red={i_wr_miss_red}\n')
# i_wr_mr_red = [float(m/a)*100 for m,a in zip(i_wr_miss_red, i_wr_access_red)]
# print(f'i wr mr red={i_wr_mr_red}')


In [12]:
# calculate hit
d_rd_hit_base = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(d_rd_access_base, d_rd_miss_base)]
d_rd_hit_quater = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(d_rd_access_quater, d_rd_miss_quater)]
d_rd_hit_red = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(d_rd_access_red, d_rd_miss_red)]

d_wr_hit_base = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(d_wr_access_base, d_wr_miss_base)]
d_wr_hit_quater = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(d_wr_access_quater, d_wr_miss_quater)]
d_wr_hit_red = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(d_wr_access_red, d_wr_miss_red)]

i_rd_hit_base = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(i_rd_access_base, i_rd_miss_base)]
i_rd_hit_quater = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(i_rd_access_quater, i_rd_miss_quater)]
i_rd_hit_red = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(i_rd_access_red, i_rd_miss_red)]

i_wr_hit_base = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(i_wr_access_base, i_wr_miss_base)]
i_wr_hit_quater = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(i_wr_access_quater, i_wr_miss_quater)]
i_wr_hit_red = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(i_wr_access_red, i_wr_miss_red)]



In [13]:
d_rd_miss_base_sum = [sum(l) for l in d_rd_miss_base]
d_rd_miss_quater_sum = [sum(l) for l in d_rd_miss_quater]
d_rd_miss_red_sum = [sum(l) for l in d_rd_miss_red]

d_rd_hit_base_sum = [sum(l) for l in d_rd_hit_base]
d_rd_hit_quater_sum = [sum(l) for l in d_rd_hit_quater]
d_rd_hit_red_sum = [sum(l) for l in d_rd_hit_red]

i_rd_miss_base_sum = [sum(l) for l in i_rd_miss_base]
i_rd_miss_quater_sum = [sum(l) for l in i_rd_miss_quater]
i_rd_miss_red_sum = [sum(l) for l in i_rd_miss_red]

i_rd_hit_base_sum = [sum(l) for l in i_rd_hit_base]
i_rd_hit_quater_sum = [sum(l) for l in i_rd_hit_quater]
i_rd_hit_red_sum = [sum(l) for l in i_rd_hit_red]


d_wr_miss_base_sum = [sum(l) for l in d_wr_miss_base]
d_wr_miss_quater_sum = [sum(l) for l in d_wr_miss_quater]
d_wr_miss_red_sum = [sum(l) for l in d_wr_miss_red]

d_wr_hit_base_sum = [sum(l) for l in d_wr_hit_base]
d_wr_hit_quater_sum = [sum(l) for l in d_wr_hit_quater]
d_wr_hit_red_sum = [sum(l) for l in d_wr_hit_red]

i_wr_miss_base_sum = [sum(l) for l in i_wr_miss_base]
i_wr_miss_quater_sum = [sum(l) for l in i_wr_miss_quater]
i_wr_miss_red_sum = [sum(l) for l in i_wr_miss_red]

i_wr_hit_base_sum = [sum(l) for l in i_wr_hit_base]
i_wr_hit_quater_sum = [sum(l) for l in i_wr_hit_quater]
i_wr_hit_red_sum = [sum(l) for l in i_wr_hit_red]


In [14]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=2,
    subplot_titles=("data tlb read", "data tlb write", "inst tlb read", "inst tlb write"))

In [15]:
# fig1
# group1: base
fig.add_trace(
    go.Bar(
        name='hit',
        x=o_arr,
        y=d_rd_hit_base_sum, 
        offsetgroup=0,
        legendgroup="base",  # this can be any string, not just "group"
        legendgrouptitle_text="base",
        marker_pattern_shape="",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=1, 
    col=1
)
fig.add_trace(
    go.Bar(
        name='miss',
        marker_pattern_shape="",
        x=o_arr,
        y=d_rd_miss_base_sum, 
        offsetgroup=0, base=d_rd_hit_base_sum,
        legendgroup="base",
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=1, 
    col=1
)

# group2: quater
fig.add_trace(
    go.Bar(
        name='hit',
        x=o_arr,
        y=d_rd_hit_quater_sum, 
        offsetgroup=1,
        legendgroup="quater",
        legendgrouptitle_text="quater",
        marker_pattern_shape="x",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=1, 
    col=1
)
fig.add_trace(
    go.Bar(
        name='miss',
        marker_pattern_shape="x",
        x=o_arr,
        y=d_rd_miss_quater_sum, 
        offsetgroup=1, base=d_rd_hit_quater_sum,
        legendgroup="quater",
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=1, 
    col=1
)

# group3: red
fig.add_trace(
    go.Bar(
        name='hit',
        x=o_arr,
        y=d_rd_hit_red_sum, 
        offsetgroup=2,
        legendgroup="fastAccess",
        legendgrouptitle_text="fastAccess",
        marker_pattern_shape="+",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=1, 
    col=1
)
fig.add_trace(
    go.Bar(
        name='miss',
        marker_pattern_shape="+",
        x=o_arr,
        y=d_rd_miss_red_sum, 
        offsetgroup=2, base=d_rd_hit_red_sum,
        legendgroup="fastAccess",
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=1, 
    col=1
)

In [16]:
# fig2
# group1: base
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=d_wr_hit_base_sum, 
        offsetgroup=3,
        marker_pattern_shape="",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=1, 
    col=2
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="",
        x=o_arr,
        y=d_wr_miss_base_sum, 
        offsetgroup=3, base=d_wr_hit_base_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=1, 
    col=2
)

# group2: quater
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=d_wr_hit_quater_sum, 
        offsetgroup=4,
        marker_pattern_shape="x",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=1, 
    col=2
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="",
        x=o_arr,
        y=d_wr_miss_quater_sum, 
        offsetgroup=4, base=d_wr_hit_quater_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=1, 
    col=2
)

# group3: red
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=d_wr_hit_red_sum, 
        offsetgroup=5,
        marker_pattern_shape="+",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=1, 
    col=2
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="",
        x=o_arr,
        y=d_wr_miss_red_sum, 
        offsetgroup=5, base=d_wr_hit_red_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=1, 
    col=2
)

In [17]:
# fig3
# group1: base
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=i_rd_hit_base_sum, 
        offsetgroup=6,
        marker_pattern_shape="",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=2, 
    col=1
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="",
        x=o_arr,
        y=i_rd_miss_base_sum, 
        offsetgroup=6, base=i_rd_hit_base_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=2, 
    col=1
)

# group2: quater
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=i_rd_hit_quater_sum, 
        offsetgroup=7,
        marker_pattern_shape="x",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=2, 
    col=1
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="x",
        x=o_arr,
        y=i_rd_miss_quater_sum, 
        offsetgroup=7, base=i_rd_hit_quater_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=2, 
    col=1
)

# group3: red
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=i_rd_hit_red_sum, 
        offsetgroup=8,
        marker_pattern_shape="+",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=2, 
    col=1
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="+",
        x=o_arr,
        y=i_rd_miss_red_sum, 
        offsetgroup=8, base=i_rd_hit_red_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=2, 
    col=1
)

In [18]:
# fig4
# group1: base
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=i_wr_hit_base_sum, 
        offsetgroup=6,
        marker_pattern_shape="",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=2, 
    col=2
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="",
        x=o_arr,
        y=i_wr_miss_base_sum, 
        offsetgroup=6, base=i_wr_hit_base_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=2, 
    col=2
)

# group2: quater
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=i_wr_hit_quater_sum, 
        offsetgroup=7,
        marker_pattern_shape="x",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=2, 
    col=2
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="x",
        x=o_arr,
        y=i_wr_miss_quater_sum, 
        offsetgroup=7, base=i_wr_hit_quater_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=2, 
    col=2
)

# group3: red
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=i_wr_hit_red_sum, 
        offsetgroup=8,
        marker_pattern_shape="+",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=2, 
    col=2
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="+",
        x=o_arr,
        y=i_wr_miss_red_sum, 
        offsetgroup=8, base=i_wr_hit_red_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=2, 
    col=2
)

In [19]:
fig.update_layout(title_text="tlb access", 
                  bargroupgap=0.2,
                  height=800, width=1600,
                 )
fig.show()

In [20]:
metric = 'bin_size_rd'
bin_rd = [get_list(o, metric) for o in outdir_hist if '_2' in o]
print(bin_rd)

metric = 'bin_size_wr'
bin_wr = [get_list(o, metric) for o in outdir_hist if '_2' in o]
print(bin_wr)

[[64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64], [256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256], [512, 1024, 512, 1024, 512, 1024, 512, 1024, 512, 1024, 512, 1024, 512, 512, 1024, 512, 1024, 512, 1024, 512, 1024, 512, 1024, 1024, 512, 1024, 512, 1024, 512, 1024, 512, 1024, 512, 1024, 512, 512, 1024, 512, 1024, 512, 1024, 512, 1024, 512, 1024, 1024, 512, 1024, 512, 1024, 512, 1024, 512, 1024, 512, 1024, 512, 512, 1024, 512, 1024, 1024, 512, 1024], [256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256,

In [21]:
metric = 'sd_t'
sd_t = [get_list(o, metric) for o in outdir_hist if '_2' in o]
# print(f'sd_t ={sd_t}')
sd_t_sum = [sum(arr) for arr in sd_t]

metric = 'sd_rd_0'
sd_rd_0 = [get_list(o, metric) for o in outdir_hist if '_2' in o]
assert len(sd_rd_0[0]) == 64
# sd_rd_0_pc = [[float(ii/tt)*100 for ii, tt in zip(i, t)] for i,t in zip(sd_rd_0, sd_t)]
# print(f'sd_rd_0 percent ={sd_rd_0_pc}')
sd_rd_0_sum = [sum(arr) for arr in sd_rd_0]
sd_rd_0_mean = [float(i/t*100) for i,t in zip(sd_rd_0_sum, sd_t_sum)]

metric = 'sd_wr_0'
sd_wr_0 = [get_list(o, metric) for o in outdir_hist if '_2' in o]
assert len(sd_wr_0[0]) == 64
# sd_rd_0_pc = [[float(ii/tt)*100 for ii, tt in zip(i, t)] for i,t in zip(sd_wr_0, sd_t)]
# print(f'sd_wr_0 percent ={sd_wr_0_pc}')
sd_wr_0_sum = [sum(arr) for arr in sd_wr_0]
sd_wr_0_mean = [float(i/t*100) for i,t in zip(sd_wr_0_sum, sd_t_sum)]


metric = 'sd_inf'
sd_inf = [get_list(o, metric) for o in outdir_hist if '_2' in o]
assert len(sd_inf[0]) == 64
# sd_inf_pc = [[float(ii/tt)*100 for ii, tt in zip(i, t)] for i,t in zip(sd_inf, sd_t)]
# print(f'sd_inf pc ={sd_inf_pc}')
sd_inf_sum = [sum(arr) for arr in sd_inf]
sd_inf_mean = [float(i/t*100) for i,t in zip(sd_inf_sum, sd_t_sum)]

print(bench)
print(sd_rd_0_mean, sd_wr_0_mean, sd_inf_mean)

['blackscholes', 'bodytrack', 'fluidanimate', 'swaptions', 'raytrace']
[42.49138440685787, 93.0646617458755, 27.116469102408875, 74.32189119242221, 28.027290768376496] [5.24260645124079, 2.817434160280812, 9.043343554373202, 16.03035362517777, 4.14112113207062] [21.446554483713285, 0.33181526160058156, 5.268446963744031, 2.7684676792240945, 29.82674476399646]


In [22]:
sd_mean = [sum(x) for x in zip(sd_rd_0_mean, sd_wr_0_mean)]
print(sd_mean)

[47.73399085809866, 95.88209590615631, 36.15981265678208, 90.35224481759998, 32.16841190044712]
